# Generating structured mesh with numpy

In [2]:
import argiope as ag
import numpy as np
import pandas as pd
import numba

In [3]:
@numba.jit
def _make_conn(shape):
    """
    Connectivity builder using Numba for speed boost.
    """
    shape = np.array(shape)
    Ne = shape.prod()
    if len(shape) == 2:
        nx, ny = np.array(shape) +1 
        conn = np.zeros((Ne, 4), dtype = np.int32)
        counter = 0
        pattern = np.array([0,1,1+nx,nx])
        for j in range(shape[1]):
            for i in range(shape[0]):
                conn[counter] = pattern + 1 + i + j*nx
                counter += 1
        
    if len(shape) == 3:
        nx, ny, nz  = np.array(shape) +1 
        conn = np.zeros((Ne, 8), dtype = np.int32)
        counter = 0
        pattern = np.array([0,1,1+nx,nx,nx*ny,1+nx*ny,1+(nx+1)*ny,(nx+1)*ny])
        for k in range(shape[2]):
            for j in range(shape[1]):
                for i in range(shape[0]):
                    conn[counter] = pattern + 1 + i + j*nx+ k*nx*ny
                    counter += 1
    return conn   

def StructuredMesh(shape = (2,2,2), dim = (1.,1.,1.)):
    """
    Returns a structured mesh.
    
    :arg shape: 2 or 3 integers (eg: shape = (10, 10, 10)).
    :type shape: tuple
    :arg dim: 2 or 3 floats (eg: dim = (4., 2., 1.))
    :type dim: tuple
        
    """
    # PREPROCESSING
    shape = np.array(shape)
    dim   = np.array(dim) 
    Ne = shape.prod()
    Nn = (shape + 1).prod()
    # LABELS
    nindex = np.arange(Nn) + 1
    eindex = np.arange(Ne) + 1
    # COORDINATES
    coords = [ np.linspace(0., dim[i], shape[i] + 1) for i in range(len(shape))]
    coords = np.array(np.meshgrid(*coords))
    coords = np.array([c.swapaxes(0,1).flatten("F") for c in coords]).T
    if len(shape) == 2:
        c = coords
        coords = np.zeros((Nn, 3))
        coords[:, :2] = c  
    # CONNECTIVITY    
    conn = _make_conn(shape)
    # MESH INSTANCE
    mesh = ag.mesh.Mesh(nlabels = nindex,
                        coords  = coords,
                        elabels = eindex,
                        conn = conn,)
    if len(shape) == 2: mesh.elements[("type", "argiope")] = "quad4"
    if len(shape) == 3: mesh.elements[("type", "argiope")] = "hexa8"    
    return mesh



In [5]:
%timeit mesh = StructuredMesh(shape = (100,100,10))

72.9 ms ± 1.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
mesh = StructuredMesh(shape = (100,10,10))
mesh.elements

In [ ]:
nindex = np.arange(1, Nn + 1)
eindex = np.arange(1, Ne + 1)

In [ ]:
#nindex = np.arange(1, Nn + 1).reshape(shape + 1)
nx, ny, nz  = np.array(shape) +1 
conn = np.zeros((Ne, 8), dtype = np.int32)
counter = 0
pattern = np.array([0,1,1+nx,nx,nx*ny,1+nx*ny,1+(nx+1)*ny,(nx+1)*ny])
for k in range(shape[2]):
    for j in range(shape[1]):
        for i in range(shape[0]):
            conn[counter] = pattern + 1 + i + j*nx+ k*nx*ny
            counter += 1
conn


In [ ]:
nindex = np.arange(1, Nn + 1).reshape(shape[::-1] + 1)
nindex[1, 0, 0]    

In [ ]:
pd.DataFrame(coords, index = nindex.flatten())

In [ ]:
'''
# CONNECTIVITY
nl = nlabels
if len(Ne) == 2:
    conn = np.array([
           nl[  :-1,  :-1],
           nl[  :-1, 1:  ],
           nl[ 1:  , 1:  ],
           nl[ 1:  ,  :-1],
           ]).swapaxes(0,2).swapaxes(0,1).reshape(Ne.prod(), 4)
    coords = np.array([ C.flatten() for C in Coords] + [ np.zeros((Ne+1).prod())]).T
    etypes = ["quad4" for i in range(Ne.prod()) ]
    
    
if len(Ne) == 3:
    nl = nl#.swapaxes(1,2).T
    conn = np.array([
           nl[  :-1,  :-1,  :-1].flatten(),
           nl[  :-1, 1:  ,  :-1].flatten(),
           nl[ 1:  , 1:  ,  :-1].flatten(),
           nl[ 1:  ,  :-1,  :-1].flatten(),
           nl[  :-1,  :-1, 1:  ].flatten(),
           nl[  :-1, 1:  , 1:  ].flatten(),
           nl[ 1:  , 1:  , 1:  ].flatten(),
           nl[ 1:  ,  :-1, 1:  ].flatten(),
           ]).T#.swapaxes(0,2).swapaxes(0,1).reshape(Ne.prod(), 8)
    Coords = Coords[[2,0,1]]
    coords = np.array([ C.flatten() for C in Coords]).T
    etypes = ["hexa8" for i in range(Ne.prod()) ]

conn'''

In [ ]:

coords


mesh = ag.mesh.Mesh(nlabels = nlabels.flatten(),
             coords  = coords,
             elabels = elabels.flatten(),
             conn = conn,
             types = etypes)
 

In [ ]:
mesh.elements

In [ ]:
mesh.nodes

In [ ]:
mesh.stats()

In [ ]:
np.arange(8).reshape(2,2,2).swapaxes(0,2)[ [0, 1, 1, 0, 0, 1, 1, 0] , [0, 0, 1,1, 0, 0, 1,1], [0,0,0,0,1,1,1,1] ]

In [ ]:
shape = 2,2,2
 
conn = []
for k in range(shape[2]):
    for j in range(shape[1]):
        for i in range(shape[0]):
            conn.append(
            [ i    +  j    * shape[1] +  k    * shape[1] * shape[2] + 1, 
             (i+1) +  j    * shape[1] +  k    * shape[1] * shape[2] + 1, 
             (i+1) + (j+1) * shape[1] +  k    * shape[1] * shape[2] + 1,
              i    + (j+1) * shape[1] +  k    * shape[1] * shape[2] + 1,
              i    +  j    * shape[1] + (k+1) * shape[1] * shape[2] + 1, 
             (i+1) +  j    * shape[1] + (k+1) * shape[1] * shape[2] + 1, 
             (i+1) + (j+1) * shape[1] + (k+1) * shape[1] * shape[2] + 1,
              i    + (j+1) * shape[1] + (k+1) * shape[1] * shape[2] + 1,
            ]
            )
np.array(conn)


In [ ]:
shape = 2,2,2
nx, ny, nz  = np.array(shape) +1 

i, j, k = 1, 1, 0
np.array(
[0,       1,       1+nx,        0+nx, 
 0+nx*ny, 1+nx*ny, 1+(nx+1)*ny, 0+(nx+1)*ny]) + 1 + i + j*nx + nx*ny*k

In [ ]:
#make_conn( (100,100,10) )

m = StructuredMesh(shape = (400,400), size = (2., 4., 8.))
m.elements

In [ ]:
_make_conn(shape = (1,1))

In [ ]:
m.elements[("type", "argiope", "")] = "ddd"
m.elements

In [ ]:
%%timeit
x, y, z = np.meshgrid(np.arange(100), np.arange(100), np.arange(100))
x, y, z = x.flatten(), y.flatten(), z.flatten()
d = pd.DataFrame({"x":x, "y":y, "z":z})
d.sort_values(["z", "y", "x"])

In [ ]:
%%timeit
shape = 100, 100, 100
shape = np.array(shape)
nx, ny, nz
Ne = shape.prod()
offsets = np.zeros(Ne)
counter = 0
for k in range(shape[2]):
    for j in range(shape[1]):
        for i in range(shape[0]):
            offsets[counter] = 1 + i + j*nx+ k*nx*ny
            counter += 1